# Bölüm 6: Öngörü (Adım Adım Düşünme)

- [Ders](#lesson)
- [Alıştırmalar](#exercises)
- [Oyun Alanı](#example-playground)

## Kurulum

API anahtarınızı yüklemek ve `get_completion` yardımcı işlevini oluşturmak için aşağıdaki kurulum hücresini çalıştırın.

In [1]:
# Bu satırı sadece bir kere çalıştırmanız gerekiyor
%pip install -q -U google-generativeai 

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import python's built-in regular expression library
import re
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

genai.configure(api_key="your-api-key")

# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

def get_completion(chat, system_prompt="NONE", prefill="NONE"):
  asistant = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    safety_settings={
          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
    },
    system_instruction= system_prompt,
  )

  chat_session = asistant.start_chat(
    history=[
      {
        "role": "user",
        "parts": [chat],
      },
      {
        "role": "model",
        "parts": [prefill],
      },
    ]
  )
  response = chat_session.send_message(chat)
  return response.text

C:\Users\AKE\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

## Ders

Biri sizi aniden uyandırsa ve hemen yanıtlamanız gereken birkaç karmaşık soru sormaya başlasa, ne yapardınız? Muhtemelen vereceğiniz cevapları önce **düşünmeniz** için size zaman verildiği bir senaryodaki kadar iyi cevaplar veremezdiniz. 

Peki tahmin edin başka nerede bu durum geçerli? Tabi ki Gemini'da.

**Gemini'a adım adım düşünmesi için zaman vermek bazen Gemini'ı daha doğru cevaplar vermesi için fırsat sağlar**, özellikle de karmaşık görevler ve soru-cevaplar için. Ancak, **düşünmek yalnızca yüksek sesle olduğunda anlamlıdır**. Gemini'dan düşünmesini isteyip sadece cevabı çıktılayamazsınız - bu durumda aslında düşünme gerçekleşmemiştir.

### Örnekler

Aşağıdaki promptta, bir insan için ikinci cümlesi ile ilk cümleyi alakalı sandığı senaryo görülmektedir. Ancak bu senaryoda **Gemini “ilgisiz” sözcüğünü fazla abartıyor**.

In [12]:
# Prompt
PROMPT = """Bu film eleştirisi olumlu mu yoksa olumsuz mu?

Bu film tazeliği ve özgünlüğüyle aklımı başımdan aldı. Tamamen ilgisiz bir haber olarak, ben 1900 yılından beri bir kayanın altında yaşıyormuşum."""

# Print Gemini's response
print(get_completion(PROMPT))

Film eleştirisi **olumlu**. 

"Tazeliği ve özgünlüğüyle aklımı başımdan aldı" ifadesi, eleştirmenin filmi oldukça beğendiğini ve etkilendiğini gösteriyor. "1900 yılından beri bir kayanın altında yaşıyormuşum" ifadesi ise, filmin kendisine yeni ve farklı bir şeyler sunduğunu, uzun zamandır böyle bir şey izlemediğini vurgulamak için kullandığı abartılı bir ifade. 



Gemini'ın yanıtını iyileştirmek için **Gemini'ın yanıt vermeden önce biraz daha düşünmesine izin verelim**. Bunu, Gemini'ın görevini işlemek ve düşünmek için atması gereken adımları tam anlamıyla açıklayarak yapıyoruz. Bir tutam rol ataması ile birlikte bu, Gemini'ın incelemeyi daha derinlemesine anlamasını sağlayacak.

In [13]:
# System prompt
SYSTEM_PROMPT = "Film eleştirileri konusunda bilgili bir okuyucu olarak davran. Yorumlarını buna göre yap."

# Prompt
PROMPT = """Bu değerlendirme olumlu mu yoksa olumsuz mudur? Önce her iki taraf için en iyi argümanları <pozitif-argüman> ve <negatif-argüman> XML etiketlerinde yazın, sonra cevaplayın.

Bu film tazeliği ve özgünlüğü ile aklımı başımdan aldı. Tamamen alakasız bir haber, 1900'den beri bir kayanın altında yaşıyormuşum."""

# Print Gemini's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

<positive_argument>Yazar, filmi "taze" ve "özgün" olarak nitelendirerek, filmin kendisini derinden etkilediğini belirten güçlü bir olumlu duygu ("aklımı başımdan aldı") ifade ediyor.</positive_argument>
<negative_argument>Yazarın aşinalık eksikliğine dair yorumu, incelemenin öznel deneyimlerine odaklandığını ve filmin nesnel kalitesine dair çok az fikir verdiğini gösterebilir.</negative_argument>

Genel olarak, bu inceleme **olumlu** olarak kabul edilir. Aşinalık eksikliğine ilişkin yorum biraz belirsiz olsa da, incelemenin ezici tonu, yazarın filmi son derece yenilikçi ve etkileyici bulduğunu açıkça ortaya koymaktadır. 



**Gemini bazen sıralamaya karşı da duyarlıdır**. Yapcağımız bu örnek, Gemini'ın nüanslı metinleri anlama becerisinin sınırındadır ve önceki örnekteki argümanların sırasını değiştirdiğimizde, negatif ilk ve pozitif ikinci olacak şekilde, bu Gemini'ın genel değerlendirmesini pozitif olarak değiştirir.

Çoğu durumda (ama hepsinde değil, yeterince kafa karıştırıcı bir şekilde), **Gemini'ın iki seçenekten ikincisini seçme olasılığı daha yüksektir**, çünkü muhtemelen web'den aldığı eğitim verilerinde ikinci seçeneklerin doğru olma olasılığı daha yüksek olmuştur.

In [15]:
# Prompt
PROMPT = """Bu değerlendirme olumsuz mu yoksa olumlu mudur? Önce her iki taraf için en iyi argümanları <positive-argument> ve <negative-argument> XML etiketlerinde yaz, sonra cevapla.

Bu film tazeliği ve özgünlüğü ile aklımı başımdan aldı. Tamamen alakasız bir haber, 1900'den beri bir kayanın altında yaşıyormuşum."""

# Print Gemini's response
print(get_completion(PROMPT))

<positive-argument>Değerlendirmen, filmin tazeliği ve özgünlüğü karşısında çok etkilendiğini açıkça belirterek güçlü bir olumlu duygu ifade ediyor. "Aklımı uçurdu" ifadesi, filmin yaratıcılığına dair coşkulu bir övgüdür.</positive-argument>
<negative-argument>1900'den beri bir kayanın altında yaşadığını belirten ikinci cümle, yorumcunun modern filmler ve neyin "taze" veya "özgün" olarak kabul edilebileceği konusundaki anlayışını sorguluyor. Bu, övgülerinin samimiyetine dair bir ironi veya şüphe unsuru katıyor.</negative-argument>

Genel olarak, değerlendirme **olumlu** olarak kabul edilir. Güçlü olumlu duyguya rağmen, yorumcunun sinematik deneyim eksikliğine dair ironik yorumu göz ardı edilemez. 



**Gemini'ın düşünmesine izin vermek, Gemini'ın cevabını yanlıştan doğruya çevirebilir**. Gemini'ın hata yaptığı birçok durumda bunu düzeltmek bu kadar kadar basit!

Gemini'dan düşünmesini istemenin bunu nasıl düzeltebileceğini görmek için Gemini'ın cevabının yanlış olduğu bir örnek üzerinden gidelim.

In [18]:
# Prompt
PROMPT = "Başrolünde 1956 yılında doğmuş bir aktörün oynadığı ünlü bir film söyle."

# Print Gemini's response
print(get_completion(PROMPT))

1956 doğumlu bir aktörün başrol oynadığı ünlü bir film **Top Gun**'dır (1986), başrolünde **Tom Cruise** oynuyor. 



Tom Cruise 1962 doğumlu bir aktördür (Not: tekrar çalıştırıldığında farklı cevaplar verebilir ancak yüksek ihtimalle yanlış verecektir). Gemini'dan bu kez `<brainstorm>` etiketleri içinde adım adım düşünmesini isteyerek yaptığı bu hatayı düzeltelim.

In [19]:
# Prompt
PROMPT = "Başrolünde 1956 yılında doğmuş bir aktörün oynadığı ünlü bir film söyle. Önce <brainstorm> etiketleri içinde bazı aktörler ve doğum yılları hakkında beyin fırtınası yap, ardından cevabını ver."

# Print Gemini's response
print(get_completion(PROMPT))

<brainstorm>
* **Tom Hanks** (1956 doğumlu)
* **John Travolta** (1954 doğumlu)
* **Mel Gibson** (1956 doğumlu)
* **Bruce Willis** (1955 doğumlu)
</brainstorm>

Başrolünde 1956 yılında doğmuş bir aktörün oynadığı ünlü bir film **Cesur Yürek**'tir. Mel Gibson hem yönetmenliğini hem de başrolünü üstlendiği filmde rol almıştır. 



---

## Alıştırmalar
- [Alıştırma 6.1 - Email Sınıflandırma](#exercise-61---classifying-emails)
- [Alıştırma 6.2 - Email Sınıflandırma Formatı](#exercise-62---email-classification-formatting)

### Alıştırma 6.1 - Email Sınıflandırma
Bu alıştırmada, Gemini'a e-postaları aşağıdaki kategorilere ayırması talimatını vereceğiz:										
- (A) Satış öncesi soru
- (B) Kırık veya kusurlu ürün
- (C) Faturalama sorusu
- (D) Diğer (lütfen açıklayınız)

Alıştırmanın ilk bölümü için `PROMPT` ifadesini **Gemini'ın doğru sınıflandırmayı ve SADECE sınıflandırmayı çıktılaması** şeklinde değiştirin. Cevabınız **doğru seçeneğin harfini (A - D) parantez içinde ve kategori adıyla birlikte içermelidir**.

E-postanın hangi kategori altında sınıflandırılması gerektiğini öğrenmek için `EMAILS` listesindeki her e-postanın yanındaki yorumlara bakın.

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Lütfen bu e-postayı yeşil ya da mavi olarak sınıflandırın: {email}"""

# Prefill for Gemini's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Gemini's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Gemini's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Gemini's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Gemini's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ NOTLANDIRMA ------------------------------------------")
    print("Bu alıştırma doğru bir şekilde çözülmüştür:", grade, "\n\n\n\n\n\n")

❓ Bir ipucu istiyorsanız, aşağıdaki hücreyi çalıştırın!

In [ ]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

Hala takıldınız mı? Örnek bir çözüm için aşağıdaki hücreyi çalıştırın.						

In [ ]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)

### Alıştırma 6.2 - Email Sınıflandırma Formatı
Bu alıştırmada, tam olarak istediğimiz şekilde biçimlendirilmiş bir yanıt elde etmek için yukarıdaki komut isteminin çıktısını iyileştireceğiz. 

Gemini'ın doğru sınıflandırmanın harfini `<answer><answer>` etiketleri içine SADECE sarmasını sağlamak için en sevdiğiniz çıktı biçimlendirme tekniğini kullanın. Örneğin, ilk e-postanın cevabı tam olarak `<answer>B<answer>` dizesini içermelidir.

Her e-posta için hangi harf kategorisinin doğru olduğunu unutursanız, `EMAILS` listesindeki her e-postanın yanındaki yorumlara bakın.

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Lütfen bu e-postayı yeşil ya da mavi olarak sınıflandırın: {email}"""

# Prefill for Gemini's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Gemini's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Gemini's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Gemini's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Gemini's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ NOTLANDIRMA ------------------------------------------")
    print("Bu alıştırma doğru bir şekilde çözülmüştür:", grade, "\n\n\n\n\n\n")

❓ Bir ipucu istiyorsanız, aşağıdaki hücreyi çalıştırın!

In [ ]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

### Tebrikler!

Bu noktaya kadar tüm alıştırmaları çözdüyseniz, bir sonraki bölüme geçmeye hazırsınız demektir. İyi çalışmalar!

---

## Oyun Alanı

Burası, bu derste gösterilen ipucu örneklerini özgürce deneyebileceğiniz ve Gemini'ın yanıtlarını nasıl etkileyebileceğini görmek için ipuçlarını değiştirebileceğiniz bir alandır.

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Gemini's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Gemini's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Gemini's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Gemini's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Gemini's response
print(get_completion(PROMPT))